In [183]:
import pandas as pd
import httpx
import time
from datetime import datetime, timedelta

In [ ]:
class Straddle:
    def __init__(self):
        self.headers = {'Accept': 'application/json'}
        self.url = "https://api.india.delta.exchange/v2"
    

In [207]:
def get_candle(symbol, end, days, resolution='5m'):
    start = end - timedelta(days=days)
    start = (time.mktime(start.timetuple()))+19800
    end = (time.mktime(end.timetuple()))+19800
    headers = {'Accept': 'application/json'}
    r  = httpx.get('https://api.india.delta.exchange/v2/history/candles', params={
        'resolution': resolution,  'symbol': symbol,  'start': str(start),  'end': str(end)}, headers = headers)
    df = pd.DataFrame(r.json()["result"])
    df["date"] = pd.to_datetime(df['time'], unit='s') + pd.Timedelta(hours=5, minutes=30)
    df = df.sort_values(by='time', ascending=True)
    df = df.reset_index()
    df = calculate_heikin_ashi(df,diff=1)
    df['ema'] = df['HA_Close'].ewm(span=28, adjust=False).mean()
    return df

In [149]:
def get_strick(sybmol,end,str_diff):
    now = datetime.now()
    expiry = ""
    if now.hour >= 17 or (now.hour == 17 and now.minute >= 29):
        now += timedelta(days=1)
        expiry = f"{str(now.day).zfill(2)}{str(now.month).zfill(2)}{str(now.year)[2:]}"
    else:
        expiry = f"{str(now.day).zfill(2)}{str(now.month).zfill(2)}{str(now.year)[2:]}"
    candle = get_candle(sybmol, end=end, days=1)
    ltp = candle["close"].iloc[-1]
    atm = round(ltp / str_diff) * str_diff
    CE = f"C-BTC-{atm-str_diff}-{expiry}"
    PE = f"P-BTC-{atm+str_diff}-{expiry}"
    return {"ce": CE, "pe": PE, "atm": atm}

In [71]:
def determine_ha_candle_type(data, diff):
    price_diff = (abs(data['HA_Close'] - data['HA_Open']) / 2) if diff == 0 else -1
    #  or price_diff > 0.1)
    if data['HA_Close'] > data['HA_Open'] and data['HA_Open'] == data['HA_Low']:
        return 1
    elif data['HA_Close'] < data['HA_Open'] and data['HA_Open'] == data['HA_High']:
        return -1
    else:
        return 0


def calculate_heikin_ashi(df, diff=0):
    # Initialize the Heikin-Ashi columns
    df['HA_Open'] = 0.0
    df['HA_High'] = 0.0
    df['HA_Low'] = 0.0
    df['HA_Close'] = 0.0

    # Calculate the first Heikin-Ashi candle
    df.at[df.index[0], 'HA_Open'] = (df['open'][0] + df['close'][0]) / 2
    df.at[df.index[0], 'HA_Close'] = (df['open'][0] + df['high'][0] + df['low'][0] + df['close'][0]) / 4
    df.at[df.index[0], 'HA_High'] = df['high'][0]
    df.at[df.index[0], 'HA_Low'] = df['low'][0]

    # Calculate the rest of the Heikin-Ashi candles
    for i in range(1, len(df)):
        df.at[df.index[i], 'HA_Open'] = (df['HA_Open'][i - 1] + df['HA_Close'][i - 1]) / 2
        df.at[df.index[i], 'HA_Close'] = (df['open'][i] + df['high'][i] + df['low'][i] + df['close'][i]) / 4
        df.at[df.index[i], 'HA_High'] = max(df['high'][i], df['HA_Open'][i], df['HA_Close'][i])
        df.at[df.index[i], 'HA_Low'] = min(df['low'][i], df['HA_Open'][i], df['HA_Close'][i])
    df["HA_Close"] = round(df["HA_Close"], 2)
    df["HA_Open"] = round(df["HA_Open"], 2)
    df["HA_High"] = round(df["HA_High"], 2)
    df["HA_Low"] = round(df["HA_Low"], 2)
    df['Candle_Type'] = df.apply(determine_ha_candle_type,diff=diff, axis=1)
    return df

In [217]:
def get_symbol(sybmol, diff, end):
    stricks= get_strick(sybmol,end=end, str_diff=diff)
    ce_ltp = stricks.get("ce")
    pe_ltp = stricks.get("pe")
    atm = stricks.get("atm")
    print("stricks",stricks)
    # ce_candle = get_candle(ce_ltp, end, days=2, resolution="15m")
    # HA_close = ce_candle["HA_Close"].iloc[-1]
    # HA_High = ce_candle["HA_High"].iloc[-1]
    # ema = ce_candle["ema"].iloc[-1]
    # sl = HA_High
    # if HA_close < ema:
    #    return{"HA_close":HA_close,"ema":ema, "sl":sl,"symbol":ce_ltp}
    # pe_candle = get_candle(pe_ltp, end, days=2, resolution="15m")
    # HA_close = pe_candle["HA_Close"].iloc[-1]
    # HA_High = pe_candle["HA_High"].iloc[-1]
    # ema = pe_candle["ema"].iloc[-1]
    # sl = HA_High
    # if HA_close < ema:
    #    return{"HA_close":HA_close,"ema":ema, "sl":sl, "symbol":pe_ltp}

    return None

In [225]:
def monitor():
    pnl = {}
    end = datetime(year=2025, month=1, day=1, hour=12, minute=00)
    symbol = None
    while True:
        if symbol is None:
            symbol = get_symbol("BTCUSD",diff=200, end=end)
            
        
        time.sleep(10)
        end = end + timedelta(minutes=15)
        if end > datetime(year=2025, month=1, day=2, hour=12, minute=00):
            break
    

In [227]:
monitor()

stricks {'ce': 'C-BTC-93800-310325', 'pe': 'P-BTC-94200-310325', 'atm': 94000}
None
stricks {'ce': 'C-BTC-93800-310325', 'pe': 'P-BTC-94200-310325', 'atm': 94000}
None
stricks {'ce': 'C-BTC-93800-310325', 'pe': 'P-BTC-94200-310325', 'atm': 94000}
None
stricks {'ce': 'C-BTC-94000-310325', 'pe': 'P-BTC-94400-310325', 'atm': 94200}
None
stricks {'ce': 'C-BTC-94000-310325', 'pe': 'P-BTC-94400-310325', 'atm': 94200}
None
stricks {'ce': 'C-BTC-94000-310325', 'pe': 'P-BTC-94400-310325', 'atm': 94200}
None
stricks {'ce': 'C-BTC-94200-310325', 'pe': 'P-BTC-94600-310325', 'atm': 94400}
None
stricks {'ce': 'C-BTC-94000-310325', 'pe': 'P-BTC-94400-310325', 'atm': 94200}
None
stricks {'ce': 'C-BTC-94000-310325', 'pe': 'P-BTC-94400-310325', 'atm': 94200}
None
stricks {'ce': 'C-BTC-94600-310325', 'pe': 'P-BTC-95000-310325', 'atm': 94800}
None
stricks {'ce': 'C-BTC-94600-310325', 'pe': 'P-BTC-95000-310325', 'atm': 94800}
None
stricks {'ce': 'C-BTC-94400-310325', 'pe': 'P-BTC-94800-310325', 'atm': 9460

In [223]:
d = datetime(year=2025, month=1, day=1, hour=17, minute=30)
print(d, d+timedelta(minutes=15))

2025-01-01 17:30:00 2025-01-01 17:45:00


In [237]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt

# Step 1: Fetch TCS Stock Data
df = yf.download("TCS.NS", start="2022-01-01", end="2025-01-01")
print(df)
# Step 2: Calculate Supertrend
# You can modify length (ATR period) and multiplier
supertrend = ta.supertrend(high=df['High'], low=df['Low'], close=df['Close'], length=10, multiplier=3)

# Step 3: Merge Supertrend into DataFrame
df = df.join(supertrend)

# Step 4: Plot Supertrend
plt.figure(figsize=(14, 6))
plt.plot(df['Close'], label='Close Price', color='blue')
plt.plot(df['SUPERT_10_3.0'], label='Supertrend', color='green')
plt.fill_between(df.index, df['SUPERT_10_3.0'], df['Close'], where=(df['SUPERTd_10_3.0'] == 1), color='green', alpha=0.2)
plt.fill_between(df.index, df['SUPERT_10_3.0'], df['Close'], where=(df['SUPERTd_10_3.0'] == -1), color='red', alpha=0.2)
plt.title('TCS Supertrend Indicator')
plt.legend()
plt.grid()
plt.show()

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TCS.NS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Empty DataFrame
Columns: [(Adj Close, TCS.NS), (Close, TCS.NS), (High, TCS.NS), (Low, TCS.NS), (Open, TCS.NS), (Volume, TCS.NS)]
Index: []


TypeError: 'NoneType' object is not iterable